In [1]:
import twitter_creds
import requests

import base64
from pprint import pprint

In [2]:
def get_bearer_header():
    uri_token_endpoint = 'https://api.twitter.com/oauth2/token'
    key_secret = f"{twitter_creds.consumer_key}:{twitter_creds.consumer_key_secret}".encode('ascii')
    b64_encoded_key = base64.b64encode(key_secret)
    b64_encoded_key = b64_encoded_key.decode('ascii')

    auth_headers = {
        'Authorization': 'Basic {}'.format(b64_encoded_key),
        'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
    }

    auth_data = {
        'grant_type': 'client_credentials'
    }

    auth_resp = requests.post(uri_token_endpoint, headers=auth_headers, data=auth_data)
    bearer_token = auth_resp.json()['access_token']

    bearer_header = {
       'Accept-Encoding': 'gzip',
       'Authorization': 'Bearer {}'.format(bearer_token),
       'oauth_consumer_key': twitter_creds.consumer_key 
    }
    return bearer_header

In [3]:
def getConversationId(id):
    uri = 'https://api.twitter.com/2/tweets?'

    params = {
        'ids':id,
        'tweet.fields':'conversation_id'
    }

    bearer_header = get_bearer_header()
    resp = requests.get(uri, headers=bearer_header, params=params)
    return resp.json()['data'][0]['conversation_id']

In [4]:

#   --url 'https://api.twitter.com/2/tweets/search/recent?query=conversation_id:1279940000004973111&tweet.fields=in_reply_to_user_id,author_id,created_at,conversation_id' \
#   --header 'Authorization: Bearer $BEARER_TOKEN'  


def getConversation(conversation_id):
    data = {}
    shown = False
    uri = 'https://api.twitter.com/2/tweets/search/recent'

    params = {
        'query': f'conversation_id:{conversation_id}',
        'tweet.fields': 'in_reply_to_user_id,author_id,created_at,conversation_id,referenced_tweets', 
    }

    bearer_header = get_bearer_header()
    token = ''
    while token != None:
        if token != '':
            params['next_token'] = token
        resp = requests.get(uri, headers=bearer_header, params=params).json()
        if not shown:
            print(resp)
            shown = True
        try:
            token = resp['meta']['next_token']
        except:
            break
        if not data:
            # print("empty")
            data['data'] = resp['data']
        else:
            # print("updating")
            data['data'].extend(resp['data'])
        # print(resp['meta']['result_count'])
    return data

In [5]:
getConversationId(1478993643516018692)

'1478993643516018692'

In [6]:
conversations = getConversation(1478993643516018692)

{'data': [{'conversation_id': '1478993643516018692', 'created_at': '2022-01-08T14:03:59.000Z', 'author_id': '1067815754202963968', 'in_reply_to_user_id': '2646263522', 'text': '@XiaomiIndia Interested. My fvt phone i know all 😍', 'referenced_tweets': [{'type': 'replied_to', 'id': '1478993643516018692'}], 'id': '1479816153274806272'}, {'conversation_id': '1478993643516018692', 'created_at': '2022-01-08T13:09:56.000Z', 'author_id': '1299574717695680513', 'in_reply_to_user_id': '2646263522', 'text': '@XiaomiIndia #Sant_Rampalji_Maharaj_App\nसभी धर्मग्रंथो का सार जानने के लिए आज ही Download करें हमारी Official App\n "Sant Rampalji Maharaj"\nGoogle Playstore से।\n👇🏻👇🏻\nDownload from Playstore https://t.co/1HBiJdbQaF', 'referenced_tweets': [{'type': 'replied_to', 'id': '1478993643516018692'}], 'id': '1479802550475841538'}, {'conversation_id': '1478993643516018692', 'created_at': '2022-01-08T12:57:29.000Z', 'author_id': '1444127718040236034', 'in_reply_to_user_id': '2646263522', 'text': '@Xia

In [8]:
conversations['data'][:10]

[{'conversation_id': '1478993643516018692',
  'created_at': '2022-01-08T14:03:59.000Z',
  'author_id': '1067815754202963968',
  'in_reply_to_user_id': '2646263522',
  'text': '@XiaomiIndia Interested. My fvt phone i know all 😍',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1478993643516018692'}],
  'id': '1479816153274806272'},
 {'conversation_id': '1478993643516018692',
  'created_at': '2022-01-08T13:09:56.000Z',
  'author_id': '1299574717695680513',
  'in_reply_to_user_id': '2646263522',
  'text': '@XiaomiIndia #Sant_Rampalji_Maharaj_App\nसभी धर्मग्रंथो का सार जानने के लिए आज ही Download करें हमारी Official App\n "Sant Rampalji Maharaj"\nGoogle Playstore से।\n👇🏻👇🏻\nDownload from Playstore https://t.co/1HBiJdbQaF',
  'referenced_tweets': [{'type': 'replied_to', 'id': '1478993643516018692'}],
  'id': '1479802550475841538'},
 {'conversation_id': '1478993643516018692',
  'created_at': '2022-01-08T12:57:29.000Z',
  'author_id': '1444127718040236034',
  'in_reply_to_user_id': '264